#### Import Packages

In [10]:
import sys
import time
import dotenv
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from skmultilearn.model_selection import IterativeStratification
import json


In [11]:
sys.path.append("../..")
from src.algorithms.multi_objective.MOEA_D_AM import MOEA_D_AM

#### Paths

In [12]:
env = dotenv.find_dotenv()

dataset_path = '/home/adel/Documents/Code/Ant-Miner/datasets/hepatitis.csv'
results_path = dotenv.get_key(env, 'SLC_RESULTS_DIR')
archive_path = dotenv.get_key(env, 'SLC_ARCHIVE_DIR')

#### Parameters

In [13]:
args = {
    "task": "single",
    "population": 50,
    "neighbors": 10,
    "groups": 5,
    "min_examples": 10,
    "max_uncovered": 10,
    "max_iter": 100,
    "gamma": 0.9,
    "delta": 0.05,
    "alpha": 1,
    "beta": 1,
    "p": 0.9,
    "pruning": 0,
    "decomposition": "weighted",
    "archive_type": "rules",
    "rulesets": None,
    "ruleset_size": 2,
    "prediction_strat": "all",
    "cross_val": 1,
    "folds": 5,
    "random_state": None,
    "runs": 20,
}

In [14]:
dataset = pd.read_csv(dataset_path)

label = ['class']
X = dataset.drop(columns=label)
y = dataset['class']

In [15]:
dataset_name = dataset_path.split('/')[-1].split('.')[0]
pruning_sfx = '_p' if args['pruning'] else '_np'
decomposition_sfx =  '_ws' if args['decomposition'] == 'weighted' else '_tch'
all_results = pd.DataFrame()
archive = {}

In [16]:
def run_once(args: dict, X, y, labels, run_id: int, archive: dict):
    """
    Executes one run of MOEA/D-AM with or without cross-validation.
    Returns the results dataframe for this run.
    """
    if args['task'] == 'single':
        results = pd.DataFrame(columns=[
            'run', 'fold', 'accuracy', 'f1_score', 'recall', 'precision',
            'hypervolume', 'nb_rules', 'term_rule_ratio', 'time'
        ])
    else:
        results = pd.DataFrame(columns=[
            'run', 'fold', 'acc', 'f1_score', 'f1_macro', 'f1_micro', 'recall', 'precision', 'hamming_loss',
            'subset_acc', 'ranking_loss', 'one_error', 'coverage', 'avg_precision', 'hypervolume', 
            'nb_rulesets', 'term_rule_ratio', 'time'
        ])
    

    if args['cross_val']:

        if args['task'] == 'single':
            sets = StratifiedKFold(n_splits=args['folds'], shuffle=True, random_state=args['random_state'])
        elif args['task'] == 'multi':
            sets = IterativeStratification(n_splits=args['folds'], order=1)
        else:
            raise ValueError("Invalid task type. Choose 'single' or 'multi'!")

        for k, (train_index, test_index) in enumerate(sets.split(X, y)):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            moea_d_aco = MOEA_D_AM(
                task=args['task'],
                population=args['population'],
                neighbors=args['neighbors'],
                groups=args['groups'],
                min_examples=args['min_examples'],
                max_uncovered=args['max_uncovered'],
                max_iter=args['max_iter'],
                p=args['p'],
                gamma=args['gamma'],
                alpha=args['alpha'],
                beta=args['beta'],
                delta=args['delta'],
                pruning=args['pruning'],
                decomposition=args['decomposition'],
                archive_type=args['archive_type'],
                rulesets=args['rulesets'],
                random_state=args['random_state']
            )

            train = pd.concat([X_train, y_train], axis=1)
            test = pd.concat([X_test, y_test], axis=1)

            start_time = time.time()
            moea_d_aco.run(dataset=train, labels=labels)
            end_time = time.time()

            hv_history = moea_d_aco.get_hv_history()
            all_points = moea_d_aco.get_all_points()

            fold_archive = moea_d_aco.ARCHIVE
            archive[f"run_{run_id}"] = archive.get(f"run_{run_id}", {})
            archive[f"run_{run_id}"][f"fold_{k+1}"] = {
                'archive': fold_archive,
                'all': all_points,
                'hv': hv_history
            }

            if args['task'] == 'single':
                acc, f1, recall, precision, nb_rules, term_rule_ratio, hypervolume = moea_d_aco.evaluate(X_test, y_test, args['prediction_strat'])

                results.loc[len(results)] = [
                    run_id, (k % args['folds']) + 1, acc*100, f1*100, recall*100, precision*100,
                    hypervolume, nb_rules, term_rule_ratio, end_time - start_time
                ]

                print (f"  Fold {k+1}/{args['folds']}: [Acc: {acc*100:.2f}, F1: {f1*100:.2f}, Recall: {recall*100:.2f}, Precision: {precision*100:.2f}, HV: {hypervolume:.2f}, Rules: {nb_rules}, TRR: {term_rule_ratio:.2f}]")
            else:
                y_pred, scores = moea_d_aco.predict(test, labels)
                metrics = moea_d_aco.evaluate_mlc(y_true=y_test.to_numpy(), y_pred=y_pred, labels=labels, scores=scores)
                results.loc[len(results)] = [
                    run_id, (k % args['folds']) + 1, metrics['acc']*100, metrics['f1_score']*100, metrics['f1_macro']*100,
                    metrics['f1_micro']*100, metrics['recall']*100, metrics['precision']*100, metrics['hamming_loss'],
                    metrics['subset_acc']*100, metrics['ranking_loss'], metrics['one_error'], metrics['coverage'], 
                    metrics['avg_precision']*100, metrics['hypervolume'], metrics['nb_rulesets'], 
                    metrics['term_rule_ratio'], end_time - start_time
                ]

                #print(f'Fold {k+1}/{args['folds']}: [Acc: {metrics['acc']*100:.2f}, F1 Score: {metrics['f1_score']*100:.2f}, F1 Macro: {metrics['f1_macro']*100:.2f}, F1 Micro: {metrics['f1_micro']*100:.4f}, Recall: {metrics['recall']*100:.4f}, Precision: {metrics['precision']*100:.4f}, Hamming Loss: {metrics['hamming_loss']:.4f}, Subset Acc: {metrics['subset_acc']*100:.2f}, Ranking Loss: {metrics['ranking_loss']:.4f}, One Error: {metrics['one_error']:.4f}, Coverage: {metrics['coverage']:.4f}, Avg Precision: {metrics['avg_precision']*100:.2f}, HV: {metrics['hypervolume']:.2f}, Rulesets: {metrics['nb_rulesets']}, TRR: {metrics['term_rule_ratio']:.2f}]')
    else:
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=args['random_state'], stratify=y
        )

        moea_d_aco = MOEA_D_AM(
            task = args['task'],
            population=args['population'],
            neighbors=args['neighbors'],
            groups=args['groups'],
            min_examples=args['min_examples'],
            max_uncovered=args['max_uncovered'],
            max_iter=args['max_iter'],
            p=args['p'],
            gamma=args['gamma'],
            alpha=args['alpha'],
            beta=args['beta'],
            delta=args['delta'],
            pruning=args['pruning'],
            decomposition=args['decomposition'],
            archive_type=args['archive_type'],
            rulesets=args['rulesets'],
            random_state=args['random_state']
        )

        train = pd.concat([X_train, y_train], axis=1)
        start_time = time.time()
        moea_d_aco.run(dataset=train, labels=['class'])
        end_time = time.time()

        acc, f1, recall, precision, term_rule_ratio = moea_d_aco.evaluate(X_test, y_test, args['prediction_strat'])
        hypervolume = moea_d_aco.get_hypervolume()
        nb_rules = len(moea_d_aco.ARCHIVE)
        time_taken = end_time - start_time

        results.loc[len(results)] = [
            run_id, 1, acc*100, f1*100, recall*100, precision*100,
            hypervolume, nb_rules, term_rule_ratio, time_taken
        ]

    return results, archive

In [17]:
for run_id in range(1, args['runs'] + 1):
    print(f"\n=== Starting Run {run_id}/{args['runs']} ===\n")
    run_results, archive = run_once(args, X, y, label, run_id, archive)
    all_results = pd.concat([all_results, run_results], ignore_index=True)

    if args['task'] == 'single':
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['accuracy'].mean():.2f} ± {run_results['accuracy'].std():.2f}, F1: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rules: {run_results['nb_rules'].mean():.2f} ± {run_results['nb_rules'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")
    else:
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['acc'].mean():.2f} ± {run_results['acc'].std():.2f}, F1 Score: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, F1 Macro: {run_results['f1_macro'].mean():.2f} ± {run_results['f1_macro'].std():.2f}, F1 Micro: {run_results['f1_micro'].mean():.2f} ± {run_results['f1_micro'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, Hamming Loss: {run_results['hamming_loss'].mean():.4f} ± {run_results['hamming_loss'].std():.4f}, Subset Acc: {run_results['subset_acc'].mean():.2f} ± {run_results['subset_acc'].std():.2f}, Ranking Loss: {run_results['ranking_loss'].mean():.4f} ± {run_results['ranking_loss'].std():.4f}, One Error: {run_results['one_error'].mean():.4f} ± {run_results['one_error'].std():.4f}, Coverage: {run_results['coverage'].mean():.4f} ± {run_results['coverage'].std():.4f}, Avg Precision: {run_results['avg_precision'].mean():.2f} ± {run_results['avg_precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rulesets: {run_results['nb_rulesets'].mean():.2f} ± {run_results['nb_rulesets'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")



=== Starting Run 1/20 ===



Running MOEA/D-AM:   0%|          | 0/100 [00:00<?, ?it/s]


ValueError: No rules found in the archive. Run the algorithm first.

In [18]:
if args['task'] == 'single':
    print(f"\nAverage results over {args['runs']} runs:")
    print(f" - Avg. Time: {all_results['time'].mean():.2f} ± {all_results['time'].std():.2f}")
    print(f" - Avg. Accuracy: {all_results['accuracy'].mean():.2f} ± {all_results['accuracy'].std():.2f}")
    print(f" - Avg. F1 Score: {all_results['f1_score'].mean():.2f} ± {all_results['f1_score'].std():.2f}")
    print(f" - Avg. Recall: {all_results['recall'].mean():.2f} ± {all_results['recall'].std():.2f}")
    print(f" - Avg. Precision: {all_results['precision'].mean():.2f} ± {all_results['precision'].std():.2f}")
    print(f" - Avg. Hypervolume: {all_results['hypervolume'].mean():.2f} ± {all_results['hypervolume'].std():.2f}")
    print(f" - Avg. Rules: {all_results['nb_rules'].mean():.2f} ± {all_results['nb_rules'].std():.2f}")
    print(f" - Avg. Term-Rule Ratio: {all_results['term_rule_ratio'].mean():.2f} ± {all_results['term_rule_ratio'].std():.2f}")


Average results over 20 runs:


KeyError: 'time'

#### save results & archive

In [ ]:
folder = "CV" if args['cross_val'] else "ALL"
dataset_name = dataset_path.split('/')[-1].split('.')[0]
pruning_sfx = '_p' if args['pruning'] else '_np'
decomposition_sfx =  '_ws' if args['decomposition'] == 'weighted' else '_tch'

# Save archive
path = f"{archive_path}/MOEA_D_AM/{folder}"

# Save aggregated results
save_path = f"{results_path}/MOEA_D_AM/{folder}"

with open(f"{path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.json", 'w') as f:
    json.dump(archive, f, indent=4)

all_results.to_csv(f"{save_path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.csv", index=False)

In [19]:
args['decomposition'] = 'tchebycheff'
dataset_name = dataset_path.split('/')[-1].split('.')[0]
pruning_sfx = '_p' if args['pruning'] else '_np'
decomposition_sfx =  '_ws' if args['decomposition'] == 'weighted' else '_tch'
all_results = pd.DataFrame()
archive = {}

for run_id in range(1, args['runs'] + 1):
    print(f"\n=== Starting Run {run_id}/{args['runs']} ===\n")
    run_results, archive = run_once(args, X, y, label, run_id, archive)
    all_results = pd.concat([all_results, run_results], ignore_index=True)

    if args['task'] == 'single':
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['accuracy'].mean():.2f} ± {run_results['accuracy'].std():.2f}, F1: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rules: {run_results['nb_rules'].mean():.2f} ± {run_results['nb_rules'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")
    else:
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['acc'].mean():.2f} ± {run_results['acc'].std():.2f}, F1 Score: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, F1 Macro: {run_results['f1_macro'].mean():.2f} ± {run_results['f1_macro'].std():.2f}, F1 Micro: {run_results['f1_micro'].mean():.2f} ± {run_results['f1_micro'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, Hamming Loss: {run_results['hamming_loss'].mean():.4f} ± {run_results['hamming_loss'].std():.4f}, Subset Acc: {run_results['subset_acc'].mean():.2f} ± {run_results['subset_acc'].std():.2f}, Ranking Loss: {run_results['ranking_loss'].mean():.4f} ± {run_results['ranking_loss'].std():.4f}, One Error: {run_results['one_error'].mean():.4f} ± {run_results['one_error'].std():.4f}, Coverage: {run_results['coverage'].mean():.4f} ± {run_results['coverage'].std():.4f}, Avg Precision: {run_results['avg_precision'].mean():.2f} ± {run_results['avg_precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rulesets: {run_results['nb_rulesets'].mean():.2f} ± {run_results['nb_rulesets'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")

if args['task'] == 'single':
    print(f"\nAverage results over {args['runs']} runs:")
    print(f" - Avg. Time: {all_results['time'].mean():.2f} ± {all_results['time'].std():.2f}")
    print(f" - Avg. Accuracy: {all_results['accuracy'].mean():.2f} ± {all_results['accuracy'].std():.2f}")
    print(f" - Avg. F1 Score: {all_results['f1_score'].mean():.2f} ± {all_results['f1_score'].std():.2f}")
    print(f" - Avg. Recall: {all_results['recall'].mean():.2f} ± {all_results['recall'].std():.2f}")
    print(f" - Avg. Precision: {all_results['precision'].mean():.2f} ± {all_results['precision'].std():.2f}")
    print(f" - Avg. Hypervolume: {all_results['hypervolume'].mean():.2f} ± {all_results['hypervolume'].std():.2f}")
    print(f" - Avg. Rules: {all_results['nb_rules'].mean():.2f} ± {all_results['nb_rules'].std():.2f}")
    print(f" - Avg. Term-Rule Ratio: {all_results['term_rule_ratio'].mean():.2f} ± {all_results['term_rule_ratio'].std():.2f}")

# Save archive
path = f"{archive_path}/MOEA_D_AM/{folder}"

# Save aggregated results
save_path = f"{results_path}/MOEA_D_AM/{folder}"

with open(f"{path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.json", 'w') as f:
    json.dump(archive, f, indent=4)

all_results.to_csv(f"{save_path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.csv", index=False)


=== Starting Run 1/20 ===



Running MOEA/D-AM:   0%|          | 0/100 [00:00<?, ?it/s]


IndexError: list index out of range

In [21]:
dataset_path = '/home/adel/Documents/Code/Ant-Miner/datasets/tic-tac-toe.csv'

dataset = pd.read_csv(dataset_path)

label = ['class']
X = dataset.drop(columns=label)
y = dataset['class']

args['decomposition'] = 'tchebycheff'
dataset_name = dataset_path.split('/')[-1].split('.')[0]
pruning_sfx = '_p' if args['pruning'] else '_np'
decomposition_sfx =  '_ws' if args['decomposition'] == 'weighted' else '_tch'
all_results = pd.DataFrame()
archive = {}

for run_id in range(1, args['runs'] + 1):
    print(f"\n=== Starting Run {run_id}/{args['runs']} ===\n")
    run_results, archive = run_once(args, X, y, label, run_id, archive)
    all_results = pd.concat([all_results, run_results], ignore_index=True)

    if args['task'] == 'single':
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['accuracy'].mean():.2f} ± {run_results['accuracy'].std():.2f}, F1: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rules: {run_results['nb_rules'].mean():.2f} ± {run_results['nb_rules'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")
    else:
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['acc'].mean():.2f} ± {run_results['acc'].std():.2f}, F1 Score: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, F1 Macro: {run_results['f1_macro'].mean():.2f} ± {run_results['f1_macro'].std():.2f}, F1 Micro: {run_results['f1_micro'].mean():.2f} ± {run_results['f1_micro'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, Hamming Loss: {run_results['hamming_loss'].mean():.4f} ± {run_results['hamming_loss'].std():.4f}, Subset Acc: {run_results['subset_acc'].mean():.2f} ± {run_results['subset_acc'].std():.2f}, Ranking Loss: {run_results['ranking_loss'].mean():.4f} ± {run_results['ranking_loss'].std():.4f}, One Error: {run_results['one_error'].mean():.4f} ± {run_results['one_error'].std():.4f}, Coverage: {run_results['coverage'].mean():.4f} ± {run_results['coverage'].std():.4f}, Avg Precision: {run_results['avg_precision'].mean():.2f} ± {run_results['avg_precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rulesets: {run_results['nb_rulesets'].mean():.2f} ± {run_results['nb_rulesets'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")

if args['task'] == 'single':
    print(f"\nAverage results over {args['runs']} runs:")
    print(f" - Avg. Time: {all_results['time'].mean():.2f} ± {all_results['time'].std():.2f}")
    print(f" - Avg. Accuracy: {all_results['accuracy'].mean():.2f} ± {all_results['accuracy'].std():.2f}")
    print(f" - Avg. F1 Score: {all_results['f1_score'].mean():.2f} ± {all_results['f1_score'].std():.2f}")
    print(f" - Avg. Recall: {all_results['recall'].mean():.2f} ± {all_results['recall'].std():.2f}")
    print(f" - Avg. Precision: {all_results['precision'].mean():.2f} ± {all_results['precision'].std():.2f}")
    print(f" - Avg. Hypervolume: {all_results['hypervolume'].mean():.2f} ± {all_results['hypervolume'].std():.2f}")
    print(f" - Avg. Rules: {all_results['nb_rules'].mean():.2f} ± {all_results['nb_rules'].std():.2f}")
    print(f" - Avg. Term-Rule Ratio: {all_results['term_rule_ratio'].mean():.2f} ± {all_results['term_rule_ratio'].std():.2f}")

# Save archive
path = f"{archive_path}/MOEA_D_AM/{folder}"

# Save aggregated results
save_path = f"{results_path}/MOEA_D_AM/{folder}"

with open(f"{path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.json", 'w') as f:
    json.dump(archive, f, indent=4)

all_results.to_csv(f"{save_path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.csv", index=False)


=== Starting Run 1/20 ===



Running MOEA/D-AM:   0%|          | 0/100 [00:00<?, ?it/s]

Running MOEA/D-AM: 100%|██████████| 100/100 [00:52<00:00,  1.91it/s]



  Fold 1/5: [Acc: 72.92, F1: 73.58, Recall: 72.92, Precision: 78.80, HV: 0.48, Rules: 10, TRR: 2.80]


Running MOEA/D-AM: 100%|██████████| 100/100 [00:42<00:00,  2.34it/s]


...

Running MOEA/D-AM: 100%|██████████| 100/100 [00:29<00:00,  3.36it/s]



  Fold 4/5: [Acc: 71.73, F1: 72.14, Recall: 71.73, Precision: 81.99, HV: 0.48, Rules: 9, TRR: 2.89]


Running MOEA/D-AM: 100%|██████████| 100/100 [00:29<00:00,  3.37it/s]


  Fold 5/5: [Acc: 82.72, F1: 83.08, Recall: 82.72, Precision: 84.74, HV: 0.48, Rules: 14, TRR: 2.86]
Run 20 completed in 30.25 seconds: [Acc: 71.72 ± 8.23, F1: 72.15 ± 8.23, Recall: 71.72 ± 8.23, Precision: 79.37 ± 5.04, HV: 0.48 ± 0.00, Rules: 9.80 ± 2.39, TRR: 2.85 ± 0.06]

Average results over 20 runs:
 - Avg. Time: 36.42 ± 5.85
 - Avg. Accuracy: 73.77 ± 8.68
 - Avg. F1 Score: 74.05 ± 8.66
 - Avg. Recall: 73.77 ± 8.68
 - Avg. Precision: 78.37 ± 6.18
 - Avg. Hypervolume: 0.68 ± 0.18
 - Avg. Rules: 9.59 ± 2.75
 - Avg. Term-Rule Ratio: 2.74 ± 0.15


NameError: name 'folder' is not defined

In [ ]:
dataset_path = '/home/adel/Documents/Code/Ant-Miner/datasets/mushrooms.csv'

dataset = pd.read_csv(dataset_path)

label = ['class']
X = dataset.drop(columns=label)
y = dataset['class']

args['decomposition'] = 'tchebycheff'
dataset_name = dataset_path.split('/')[-1].split('.')[0]
pruning_sfx = '_p' if args['pruning'] else '_np'
decomposition_sfx =  '_ws' if args['decomposition'] == 'weighted' else '_tch'
all_results = pd.DataFrame()
archive = {}

for run_id in range(1, args['runs'] + 1):
    print(f"\n=== Starting Run {run_id}/{args['runs']} ===\n")
    run_results, archive = run_once(args, X, y, label, run_id, archive)
    all_results = pd.concat([all_results, run_results], ignore_index=True)

    if args['task'] == 'single':
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['accuracy'].mean():.2f} ± {run_results['accuracy'].std():.2f}, F1: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rules: {run_results['nb_rules'].mean():.2f} ± {run_results['nb_rules'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")
    else:
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['acc'].mean():.2f} ± {run_results['acc'].std():.2f}, F1 Score: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, F1 Macro: {run_results['f1_macro'].mean():.2f} ± {run_results['f1_macro'].std():.2f}, F1 Micro: {run_results['f1_micro'].mean():.2f} ± {run_results['f1_micro'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, Hamming Loss: {run_results['hamming_loss'].mean():.4f} ± {run_results['hamming_loss'].std():.4f}, Subset Acc: {run_results['subset_acc'].mean():.2f} ± {run_results['subset_acc'].std():.2f}, Ranking Loss: {run_results['ranking_loss'].mean():.4f} ± {run_results['ranking_loss'].std():.4f}, One Error: {run_results['one_error'].mean():.4f} ± {run_results['one_error'].std():.4f}, Coverage: {run_results['coverage'].mean():.4f} ± {run_results['coverage'].std():.4f}, Avg Precision: {run_results['avg_precision'].mean():.2f} ± {run_results['avg_precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rulesets: {run_results['nb_rulesets'].mean():.2f} ± {run_results['nb_rulesets'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")

if args['task'] == 'single':
    print(f"\nAverage results over {args['runs']} runs:")
    print(f" - Avg. Time: {all_results['time'].mean():.2f} ± {all_results['time'].std():.2f}")
    print(f" - Avg. Accuracy: {all_results['accuracy'].mean():.2f} ± {all_results['accuracy'].std():.2f}")
    print(f" - Avg. F1 Score: {all_results['f1_score'].mean():.2f} ± {all_results['f1_score'].std():.2f}")
    print(f" - Avg. Recall: {all_results['recall'].mean():.2f} ± {all_results['recall'].std():.2f}")
    print(f" - Avg. Precision: {all_results['precision'].mean():.2f} ± {all_results['precision'].std():.2f}")
    print(f" - Avg. Hypervolume: {all_results['hypervolume'].mean():.2f} ± {all_results['hypervolume'].std():.2f}")
    print(f" - Avg. Rules: {all_results['nb_rules'].mean():.2f} ± {all_results['nb_rules'].std():.2f}")
    print(f" - Avg. Term-Rule Ratio: {all_results['term_rule_ratio'].mean():.2f} ± {all_results['term_rule_ratio'].std():.2f}")

# Save archive
path = f"{archive_path}/MOEA_D_AM/{folder}"

# Save aggregated results
save_path = f"{results_path}/MOEA_D_AM/{folder}"

with open(f"{path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.json", 'w') as f:
    json.dump(archive, f, indent=4)

all_results.to_csv(f"{save_path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.csv", index=False)

In [ ]:
dataset_path = '/home/adel/Documents/Code/Ant-Miner/datasets/mushrooms.csv'

dataset = pd.read_csv(dataset_path)

label = ['class']
X = dataset.drop(columns=label)
y = dataset['class']

args['decomposition'] = 'weighted'
dataset_name = dataset_path.split('/')[-1].split('.')[0]
pruning_sfx = '_p' if args['pruning'] else '_np'
decomposition_sfx =  '_ws' if args['decomposition'] == 'weighted' else '_tch'
all_results = pd.DataFrame()
archive = {}

for run_id in range(1, args['runs'] + 1):
    print(f"\n=== Starting Run {run_id}/{args['runs']} ===\n")
    run_results, archive = run_once(args, X, y, label, run_id, archive)
    all_results = pd.concat([all_results, run_results], ignore_index=True)

    if args['task'] == 'single':
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['accuracy'].mean():.2f} ± {run_results['accuracy'].std():.2f}, F1: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rules: {run_results['nb_rules'].mean():.2f} ± {run_results['nb_rules'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")
    else:
        print (f"Run {run_id} completed in {run_results['time'].mean():.2f} seconds: [Acc: {run_results['acc'].mean():.2f} ± {run_results['acc'].std():.2f}, F1 Score: {run_results['f1_score'].mean():.2f} ± {run_results['f1_score'].std():.2f}, F1 Macro: {run_results['f1_macro'].mean():.2f} ± {run_results['f1_macro'].std():.2f}, F1 Micro: {run_results['f1_micro'].mean():.2f} ± {run_results['f1_micro'].std():.2f}, Recall: {run_results['recall'].mean():.2f} ± {run_results['recall'].std():.2f}, Precision: {run_results['precision'].mean():.2f} ± {run_results['precision'].std():.2f}, Hamming Loss: {run_results['hamming_loss'].mean():.4f} ± {run_results['hamming_loss'].std():.4f}, Subset Acc: {run_results['subset_acc'].mean():.2f} ± {run_results['subset_acc'].std():.2f}, Ranking Loss: {run_results['ranking_loss'].mean():.4f} ± {run_results['ranking_loss'].std():.4f}, One Error: {run_results['one_error'].mean():.4f} ± {run_results['one_error'].std():.4f}, Coverage: {run_results['coverage'].mean():.4f} ± {run_results['coverage'].std():.4f}, Avg Precision: {run_results['avg_precision'].mean():.2f} ± {run_results['avg_precision'].std():.2f}, HV: {run_results['hypervolume'].mean():.2f} ± {run_results['hypervolume'].std():.2f}, Rulesets: {run_results['nb_rulesets'].mean():.2f} ± {run_results['nb_rulesets'].std():.2f}, TRR: {run_results['term_rule_ratio'].mean():.2f} ± {run_results['term_rule_ratio'].std():.2f}]")

if args['task'] == 'single':
    print(f"\nAverage results over {args['runs']} runs:")
    print(f" - Avg. Time: {all_results['time'].mean():.2f} ± {all_results['time'].std():.2f}")
    print(f" - Avg. Accuracy: {all_results['accuracy'].mean():.2f} ± {all_results['accuracy'].std():.2f}")
    print(f" - Avg. F1 Score: {all_results['f1_score'].mean():.2f} ± {all_results['f1_score'].std():.2f}")
    print(f" - Avg. Recall: {all_results['recall'].mean():.2f} ± {all_results['recall'].std():.2f}")
    print(f" - Avg. Precision: {all_results['precision'].mean():.2f} ± {all_results['precision'].std():.2f}")
    print(f" - Avg. Hypervolume: {all_results['hypervolume'].mean():.2f} ± {all_results['hypervolume'].std():.2f}")
    print(f" - Avg. Rules: {all_results['nb_rules'].mean():.2f} ± {all_results['nb_rules'].std():.2f}")
    print(f" - Avg. Term-Rule Ratio: {all_results['term_rule_ratio'].mean():.2f} ± {all_results['term_rule_ratio'].std():.2f}")

# Save archive
path = f"{archive_path}/MOEA_D_AM/{folder}"

# Save aggregated results
save_path = f"{results_path}/MOEA_D_AM/{folder}"

with open(f"{path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.json", 'w') as f:
    json.dump(archive, f, indent=4)

all_results.to_csv(f"{save_path}/{dataset_name}{decomposition_sfx}{pruning_sfx}_r{args['runs']}.csv", index=False)